In [1]:
import os
import sys
import pickle
import numpy as np
import torch 
from tqdm import tqdm

os.environ["CONFIG_PATH"] = "../configs/large_model.toml"
sys.path.append("../src")

import neural_net
from configuration import config
from neural_net import NeuralNet, ResidualBlock
import player_pov_helpers

""

Loaded config from file: ../configs/large_model.toml
Loading file: piece_indices
Loading file: rotation_mapping
Loading file: new_occupieds
Loading file: moves_ruled_out_for_all
Loading file: scores
Loading file: moves_ruled_out_for_player
Loading file: moves_enabled_for_player
Loading file: new_adjacents
Loading file: new_corners


''

In [2]:
BOARD_SIZE = config()["game"]["board_size"]

In [3]:
def model_size(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum(p.numel() for p in model_parameters)
    # return sum([np.prod(p.size()) for p in model_parameters])
    # return sum(p.numel() for p in model.parameters())

In [4]:
import time

def time_per_eval(num_evaluations, batch_size, model):
    num_batches = num_evaluations // batch_size
    num_evaluations = num_batches * batch_size

    random_arrays = np.random.random((num_batches, batch_size, 4, BOARD_SIZE, BOARD_SIZE))
    print("Starting...")

    start = time.perf_counter()
    for i in range(num_batches):
        occupancies = torch.from_numpy(random_arrays[i]).to(dtype=torch.float, device="mps")
        model(occupancies)
    elapsed = time.perf_counter() - start

    return elapsed / (num_batches * batch_size)

In [5]:
model = NeuralNet().to("mps")
model_size(model)
print(len(model.residual_blocks))

19


In [10]:
time_per_eval(
    num_evaluations=5000,
    batch_size=500,
    model=model,
)

Starting...


0.0003070897334022447

In [6]:
# Time per move:
times_per_eval = {}
for batch_size in tqdm(range(10, 600, 10)):
    times_per_eval[batch_size] = time_per_eval(
        num_evaluations=batch_size * 20,
        batch_size=batch_size,
        model=model,
    )

  0%|          | 0/59 [00:00<?, ?it/s]

Starting...


  2%|▏         | 1/59 [00:00<00:18,  3.11it/s]

Starting...


  3%|▎         | 2/59 [00:00<00:17,  3.32it/s]

...

 97%|█████████▋| 57/59 [02:03<00:07,  3.53s/it]

Starting...


 98%|█████████▊| 58/59 [02:06<00:03,  3.61s/it]

Starting...


100%|██████████| 59/59 [02:10<00:00,  2.22s/it]


In [7]:
size_distribution = {0: 0, 10: 0, 20: 0, 30: 0, 40: 2, 50: 0, 60: 0, 70: 2, 80: 5, 90: 0, 100: 0, 110: 8, 120: 8, 130: 9, 140: 10, 150: 16, 160: 25, 170: 23, 180: 7, 190: 0, 200: 3, 210: 1, 220: 2, 230: 6, 240: 2, 250: 8, 260: 13, 270: 7, 280: 0, 290: 18, 300: 6, 310: 2, 320: 4, 330: 2, 340: 0, 350: 2, 360: 0, 370: 4, 380: 8, 390: 8, 400: 6, 410: 4, 420: 4, 430: 3, 440: 3, 450: 1, 460: 0, 470: 2, 480: 1, 490: 0, 500: 0, 510: 1, 520: 0, 530: 0, 540: 0, 550: 0, 560: 0, 570: 0, 580: 0, 590: 0, 600: 0, 610: 0, 620: 0, 630: 0, 640: 0, 650: 0, 660: 0, 670: 0, 680: 0, 690: 0, 700: 0, 710: 0, 720: 0, 730: 0, 740: 0, 750: 0, 760: 0, 770: 0, 780: 0, 790: 0, 800: 0, 810: 0, 820: 0, 830: 0, 840: 0, 850: 0, 860: 0, 870: 0, 880: 0, 890: 0, 900: 0, 910: 0, 920: 0, 930: 0, 940: 0, 950: 0, 960: 0, 970: 0, 980: 0, 990: 0}

total_time = 0
num_evaluations = 0
for size in size_distribution:
    if size_distribution[size] == 0:
        continue
    total_time += size_distribution[size] * times_per_eval[size]
    num_evaluations += size_distribution[size]
print(total_time / num_evaluations)

0.0003870881047853177


In [8]:
time_per_eval(
    num_evaluations=5000,
    batch_size=100,
    model=model,
)

Starting...


0.0003615001916012261

In [9]:
times_per_eval

{10: 0.001602485000039451,
 20: 0.0007163511449471117,
 30: 0.0005632886816844499,
 40: 0.00048660171876690583,
 50: 0.0004494004999869503,
 60: 0.0004214411108356823,
 70: 0.00038294005929076643,
 80: 0.0003738480731226446,
 90: 0.0003639423844449791,
 100: 0.0003480814164940966,
 110: 0.0003374426704513925,
 120: 0.00032602670124712557,
 130: 0.00034013450346090114,
 140: 0.0003350290178579079,
 150: 0.0003281990833347663,
 160: 0.0003244789715608931,
 170: 0.00032740600500427025,
 180: 0.00032284310194275655,
 190: 0.00031721354184188196,
 200: 0.0003257472394980141,
 210: 0.00033658351190665384,
 220: 0.00032906508522997186,
 230: 0.00032740791673701176,
 240: 0.00034571629333489303,
 250: 0.0003564988666039426,
 260: 0.00040498254019229743,
 270: 0.00044534993833318973,
 280: 0.00037290672607403916,
 290: 0.0003526340589651854,
 300: 0.00036806953483513403,
 310: 0.00035447608871054986,
 320: 0.0003311281185960979,
 330: 0.0004576303346998016,
 340: 0.0003531430085294072,
 350: 0.

In [ ]:
import time

def time_per_eval(num_evaluations, batch_size, model):
    num_batches = num_evaluations // batch_size
    num_evaluations = num_batches * batch_size

    random_arrays = np.random.random((num_batches, batch_size, 4, BOARD_SIZE, BOARD_SIZE))
    print("Starting...")

    start = time.perf_counter()
    for i in range(num_batches):
        occupancies = torch.from_numpy(random_arrays[i]).to(dtype=torch.float, device="mps")
        model(occupancies)
    elapsed = time.perf_counter() - start

    return elapsed / (num_batches * batch_size)

In [21]:
player_pov_helpers.moves_indices_to_player_pov([625], 1)

array([820])

In [23]:
NUM_MOVES = config()["game"]["num_moves"]

universal_moves_array = np.zeros((NUM_MOVES,))
universal_moves_array[625] = 1

player_moves_array = player_pov_helpers.moves_array_to_player_pov(universal_moves_array, 1)
np.argmax(player_moves_array)

820

In [44]:
player_whose_turn_it_is = 1

# In my node, index 0 represents move 123, index 1 represents move 625, index 2 represents move 1052.
# This is purely compression, nothing to do with player rotations.
array_index_to_move_index = np.array([123, 625, 1052])

# This is the argument I pass into my NN
player_pov_valid_move_indices = player_pov_helpers.moves_indices_to_player_pov(array_index_to_move_index, player_whose_turn_it_is)
print("player_pov_valid_move_indices", player_pov_valid_move_indices)

# Use the old, trusted function to determine where universal move 625 maps to in this player's POV.
universal_moves_array = np.zeros((NUM_MOVES,))
universal_moves_array[625] = 1
nn_recommended_move_index = np.argmax(player_pov_helpers.moves_array_to_player_pov(universal_moves_array, player_whose_turn_it_is))
print("nn_recommended_move_index", nn_recommended_move_index)

# My NN says that in the player POV, move 820 is best. (That's the move which move 625 becomes)
policy_that_my_nn_returned = np.zeros((NUM_MOVES,))
policy_that_my_nn_returned[nn_recommended_move_index] = 1

# In my evaluate, I use the rotated move indices to filter the policy my NN returned.
policy_that_evaluate_returns = policy_that_my_nn_returned[player_pov_valid_move_indices]

universal_policy_that_i_save_to_my_node = policy_that_evaluate_returns
universal_policy_that_i_save_to_my_node

player_pov_valid_move_indices [ 560  820 1059]
nn_recommended_move_index 820


array([0., 1., 0.])

In [39]:
player_pov_valid_move_indices

array([ 560,  820, 1059])